# The thermodynamics of a eutectic solution

*Authors: Enze Chen (University of California, Berkeley)*

![Regular solution model](https://raw.githubusercontent.com/enze-chen/learning_modules/master/fig/eutectic_PD.png)

This is an interactive notebook for playing around with the temperature and observing the effect on the resulting free energy curves of a eutetic system (phase diagram shown above) described by the regular solution model. Thermodynamics is fairly math-heavy and a tough subject to master, but I believe having quality visualizations can have a large, positive impact on your learning. I hope you will find this notebook functional, readable, and educational.

## How to run this notebook

If you are viewing this notebook on Google Colaboratory, then all the software is already set up for you (hooray). If you want to run the notebook locally, make sure all the Python libraries in the [`requirements.txt`](https://github.com/enze-chen/learning_modules/blob/master/requirements.txt) file are installed.

In this notebook, everything is completed for you and you just have to run it. If you want to try coding yourself, you can look at other [thematically similar notebooks](https://github.com/enze-chen/learning_modules/tree/master/mse) that I've created, or delete sections of the code below. To execute each cell of the notebook and automatically advance to the next cell, press `Shift+Enter`. Or you can click `Run All` in the menu at the top. When done successfully, you'll see a widget slider pop up near the middle that you can adjust and observe the changes in the free energy curves. At the very end we will use the code to draw a eutectic phase diagram.

## Acknowledgements
I thank [Prof. Gerbrand Ceder](https://mse.berkeley.edu/people_new/ceder/) for teaching MATSCI 201A: Thermodynamics and Phase Transformations in Solids and my advisor [Prof. Mark Asta](https://mse.berkeley.edu/people_new/asta/) for suggesting this notebook, providing preliminary scripts and data, and encouraging me in my education-related pursuits. Interactivity is enabled with the [`ipywidgets`](https://ipywidgets.readthedocs.io/en/stable/) library. An interactive version of this notebook can be found online at [Google Colaboratory](https://colab.research.google.com/github/enze-chen/learning_modules/blob/master/mse/Eutectic_solution.ipynb).

## Important equations

We know from thermodynamics that in general,

$$ \Delta G = \Delta H - T \Delta S$$

where $\Delta H$ is the enthalpy, $T$ is the temperature, and $\Delta S$ is the entropy. Hopefully this is ingrained in your head by now.

In the regular solution model, we have an interaction term in the enthalpy given by the $\beta$ parameter, where

$$ \Delta H_{\text{mix}} = \beta x(1-x) $$

and $x$ is the atomic fraction of species $B$ in the $AB$ solutions. For ideal solutions, we can simply set $\beta = 0$. The entropy of mixing is given by

$$ \Delta S_{\text{mix}} = -R \left[ x \ln x + (1-x) \ln(1-x) \right] $$

Furthermore, we'll assume for this demo that the free energies of the pure solid phases are $0$ (reference).


## Assumptions
* I assumed that the two curves would be mostly convex for my optimization routine.
* I assumed that there will only be one common tangent (i.e. no "boomerang" phase diagrams).
* I assumed that self-common tangents are mostly horizontal (i.e. curve is not "tilted").

## Known issues
* It doesn't have great safeguards against index-out-of-bounds errors, which will happen if you put absurd quantities for the $\beta$ parameter. Please be gentle. ❤

## Python library imports

These are all the required Python libraries.

In [ ]:
# General libraries
import warnings
warnings.filterwarnings('ignore')  # please do as I say, not as I do

# Scientific computing libraries
import numpy as np
from scipy.misc import derivative
import matplotlib.pyplot as plt
%matplotlib inline

# Interactivity libraries
from ipywidgets import interact, interact_manual, fixed, \
                       IntSlider, FloatSlider, FloatLogSlider, \
                       Button, Layout

## Analytical functions

Here are the theoretical curves for the Gibbs free energy ($\Delta G$) of the solid and liquid phases, with data taken from a certain material $A$ and a certain material $B$. We define $x$ to be the atomic fraction of $B$, which substitutes on the $A$ sites. $\beta$ is the interaction parameter for the regular solution model, and has a default value of $0$ (ideal solution).

In [ ]:
def curve_s(x, T, beta=0):
    """This function plots the Gibbs free energy curve for the solid solution.
    
    Args:
        x (numpy.darray): An array of atomic fractions of B.
        T (float): The temperature in Kelvin.
        beta (float): The interaction parameter in J/mol.
        
    Returns:
        G_s (numpy.darray): An array of Gibbs free energy values in kJ/mol.    
    """
    S_mix = -8.314 * (np.multiply(x, np.log(x)) + np.multiply(1 - x, np.log(1 - x)))
    H_mix = beta * np.multiply(x, 1 - x)
    G_s = -T * S_mix + H_mix
    return G_s / 1000


def curve_l(x, T, beta=0):
    """This function plots the Gibbs free energy curve for the liquid solution.
    
    Args:
        x (numpy.darray): An array of atomic fractions of B.
        T (float): The temperature in Kelvin.
        beta (float): The interaction parameter in J/mol.
        
    Returns:
        G_l (numpy.darray): An array of Gibbs free energy values in kJ/mol.    
    """
    H_A, H_B = (8550, 9800)
    T_A, T_B = (1100, 1200)
    S_A, S_B = (H_A/T_A, H_B/T_B)
    G_A = H_A - T * S_A
    G_B = H_B - T * S_B
    S_mix = -8.314 * (np.multiply(x, np.log(x)) + np.multiply(1 - x, np.log(1 - x)))
    H_mix = beta * np.multiply(x, 1 - x)
    G_l = x * G_B + (1 - x) * G_A - T * S_mix + H_mix
    return G_l / 1000

## Constructing the common tangent

There are many ways one could go about doing this. I've seen people in the past do a brute force enumeration (\*shudder\*). John Dagdelen suggests a Legendre transform (nifty!). I got really excited about trying gradient descent optimization (yeet). Eventually, after much Googling and consideration, I went with something simple and relatively fast, inspired from [this StackOverflow answer](https://stackoverflow.com/a/10271179). The procedure is as follows:

1. Using `scipy`, compute a derivative of curve 1 and store this for later use.
1. Pick the minimum of curve 1 as an initial guess as the common tangent will be close to there.
1. Using the derivative and coordinates, construct the tangent line at the minimum.
1. Extend the tangent line and see if it intersects curve 2.
    1. **If so**, shift the tangent point *left* on curve 1 and continue building tangent lines of decreasing slope.
        1. Repeat step 4.
        1. As soon as an extended tangent line *misses* curve 2, **stop**.
    1. **If not**, shift the tangent point right on curve 1 and continue building tangent lines of increasing slope.
        1. Repeat step 4.
        1. As soon as an extended tangent line *intersects* curve 2, **stop**.
1. Identify where the common tangent intersects the two curves so we only plot that portion.

In [ ]:
def common_tangent(x, y1, y2, fn, T, beta=0):
    """This function calculates the common tangent of two convex curves.
    
    Args:
        x (numpy.darray): An array of atomic fractions of B.
        y1 (numpy.darray): y values for curve 1.
        y2 (numpy.darray): y values for curve 2.
        fn (function): The function that we take the derivative of.
        T (float): The temperature in Kelvin.
        beta (float): The interaction parameter in J/mol.
        
    Returns:
        line (numpy.darray): y values for the common tangent.
        idmin (int): Index of the x-coordinate of the first tangent point.
        idmax (int): Index of the x-coordinate of the second tangent point.
    """
    # Compute a derivative
    dx = 1e-3
    dy1 = derivative(func=fn, x0=x, dx=dx, args=(T, beta))

    # Make an initial guess at the minimum of curve 1
    n = len(x)
    idmin, idmax = (0, n)
    idx = np.argmin(y1)
    yp = y1[idx]
    xp = x[idx]
    dyp = dy1[idx]

    # Construct the tangent line and count intersections with curve 2
    thresh = 1
    line = dyp * x + yp - dyp * xp
    diff = np.diff(np.sign(y2 - line))
    nnz = np.count_nonzero(diff)

    # They're the same curve. Used for finding miscibility gap.
    # I'm assuming that the curve is symmetric
    if np.linalg.norm(y1 - y2) < 1e-4:
        idmin = np.argmin(y1[:int(n/2)])
        idmax = np.argmin(y1[int(n/2):]) + int(n/2)
    
    # If the tangent line intersects curve 2, shift tangent point to the left
    elif nnz >= thresh:
        while nnz >= thresh:
            idx -= 1
            # try-except to avoid an out-of-bounds error 
            try:
                yp = y1[idx]
                xp = x[idx]
                dyp = dy1[idx]
                line = dyp * x + yp - dyp * xp
                diff = np.diff(np.sign(y2 - line))
                nnz = np.count_nonzero(diff)
            except:
                break
            if diff.any():
                # Assign left and right indices of the tangent points
                # Here we do it each time because once we miss, we can't go back
                idmax = np.nonzero(diff)[0][0]
        idmin = idx

    # If the tangent line misses curve 2, shift tangent point to the right
    elif nnz < thresh:
        while nnz < thresh:
            idx += 1
            # try-except to avoid an out-of-bounds error 
            try:
                yp = y1[idx]
                xp = x[idx]
                dyp = dy1[idx]
                line = dyp * x + yp - dyp * xp
                diff = np.diff(np.sign(y2 - line))
                nnz = np.count_nonzero(diff)
            except:
                break
        # Assign left and right indices of the tangent points
        idmin = idx
        idmax = np.nonzero(diff)[0][0]
    
    # Return a tuple
    return (line, idmin, idmax)

## Widget function

Our widget will call `plot_Gx()` each time we interact with it. This function calls `curve_s()` and `curve_l()` to get the solid and liquid free energy curves, respectively. It then calls the `common_tangent()` function above to calculate the common tangent to the two curves. The rest of the function handles the plotting, where I specifically chose to lock the $y$-axis to emphasize how much $\Delta G^l$ changes as a function of temperature compared to $\Delta G^s$.

In [ ]:
def plot_Gx(T=1000, beta_s=0, beta_l=0):
    """This function is called by the widget to perform the plotting based on inputs.

    Args:
        T (float): The temperature in Kelvin.
        beta_s (float): The interaction parameter for solids in J/mol.
        beta_l (float): The interaction parameter for liquids in J/mol.

    Returns:
        None, but a pyplot is displayed.
    """
    # For the given temperature, calculate the curves and common tangent
    n = int(9e3)
    xmin, xmax = (0.001, 0.999)
    x = np.linspace(xmin, xmax, n)
    plot_sl_tangent = False
    y_s = curve_s(x, T, beta_s)
    y_l = curve_l(x, T, beta_l)

    # First check solid-solid common tangent
    line, idmin, idmax = common_tangent(x, y_s, y_s, curve_s, T, beta_s)
    lmin = np.argmin(y_l)
    if line[lmin] <= y_l[lmin]:
        ids = np.absolute([idmin, idmax])
    # There are solid-liquid common tangent(s)
    else:
        line1, idmin1, idmax1 = common_tangent(x, y_s, y_l, curve_s, T, beta_s)
        line2, idmin2, idmax2 = common_tangent(x, y_l, y_s, curve_l, T, beta_l)
        ids = np.absolute([idmin1, idmax1, idmin2, idmax2])
        plot_sl_tangent = True

    # Mostly plot settings for visual appeal
    plt.rcParams.update({'figure.figsize':(8,6), 'font.size':20, \
                         'lines.linewidth':4, 'axes.linewidth':2})
    fig, ax = plt.subplots()
    ymin, ymax = (-6.5, 5)
    ax.plot(x, y_s, c='C0', label='solid')
    ax.plot(x, y_l, c='C1', label='liquid')
    if (ids < n).all():
        if plot_sl_tangent:
            ax.annotate(s=r'$L$', xy=(0.4, -5.5))
            if ids[0] < ids[1]:
                ax.plot(x[idmin1:idmax1], line1[idmin1:idmax1], c='k', lw=5, ls='-.')
                xlow, xhigh = (x[idmin1], x[idmax1])
                ax.vlines(x=[xlow, xhigh], ymin=ymin, \
                          ymax=[line1[idmin1], line1[idmax1]], linestyles='dotted', linewidth=3)
                ax.annotate(s=r'$\alpha$', xy=(xlow/2 - 0.02, -3.8))
                ax.annotate(s=r'$\alpha+L$', xy=((xlow + xhigh)/2 - 0.04, -4.8))
            if ids[2] < ids[3]:
                ax.plot(x[idmin2:idmax2], line2[idmin2:idmax2], c='k', lw=5, ls='-.')
                xlow, xhigh = (x[idmin2], x[idmax2])
                ax.vlines(x=[xlow, xhigh], ymin=ymin, \
                          ymax=[line2[idmin2], line2[idmax2]], linestyles='dotted', linewidth=3)
                ax.annotate(s=r'$L+\beta$', xy=((xlow + xhigh)/2 - 0.04, -4.8))
                ax.annotate(s=r'$\beta$', xy=((xhigh + 1)/2 - 0.02, -3.8))            
        elif ids[0] < ids[1]:
            ax.plot(x[idmin:idmax], line[idmin:idmax], c='k', lw=5, ls='-.')
            xlow, xhigh = (x[idmin], x[idmax])
            ax.vlines(x=[xlow, xhigh], ymin=ymin, \
                      ymax=[line[idmin], line[idmax]], linestyles='dotted', linewidth=3)
            ax.annotate(s=r'$\alpha$', xy=(xlow/2 - 0.02, -3.8))
            ax.annotate(s=r'$\alpha+\beta$', xy=((xlow + xhigh)/2 - 0.05, -4.8))
            ax.annotate(s=r'$\beta$', xy=((xhigh + 1)/2 - 0.02, -3.8))
    ax.tick_params(top=True, right=True, direction='in', length=10, width=2)
    ax.set_xlim(0, 1)
    ax.set_ylim(ymin, ymax)
    ax.set_xlabel(r'$x_{B}$')
    ax.set_ylabel(r'$\Delta G$ (kJ/mol)')
    ax.set_title('Gibbs free energy at T = {} K'.format(T), fontsize=18)
    plt.legend(loc='upper right')
    plt.show()

We create each slider individually for readability and customization.

In [ ]:
mystyle = {'description_width':'160px'}
mylayout = Layout(width='450px', height='30px')

T_widget = IntSlider(value=900, min=500, max=1300, step=50, \
                     description='Temperature (K)', readout_format='d', \
                     style=mystyle, layout=mylayout, continuous_update=False)

Each time you release your mouse, the plot will automatically update! This is because we specified `continuous_update=False` in the constructor. The search can be a little slow at times, so *don't change the slider too rapidly* (watch the title to see that it matches your setting).

In [ ]:
interact(plot_Gx, T=T_widget, beta_s=fixed(20000), beta_l=fixed(10000));


--------------------------------------------

--------------------------------------------

--------------------------------------------


## Creating the phase diagram

Now that we have everything set up, it's your job to finish off the notebook. This probably sounds silly, but before taking MATSCI 201A, it did not occur to me that the data above actually correspond to a phase diagram. I guess that just wasn't emphasized in my undergraduate studies or I simply dozed off. **:shrug:**

But they do! Specifically, the common tangents indicate, for a specific $T$, the regions of two-phase coexistence. The two tangent points lie on the appropriate boundaries. Knowing this, we'll now loop through a few values of $T$ (`Ts`) to obtain a set of $x$-coordinates and then construct the phase diagram from them.

In [ ]:
# Enumerate the temperatures we want to calculate phase boundaries at. 
Ts = [600, 700, 800, 892, 900, 950, 1000, 1050, 1090, 1100, 1150, 1190]
beta_s = 20000
beta_l = 10000

# Create empty arrays to track the x-coordinates of the boundaries
liquidus = []
solvus1 = []
solvus2 = []

# Same x-axis as before
n = int(1e4)
xmin, xmax = (0.001, 0.999)
x = np.linspace(xmin, xmax, n)

plot_sl = False
for T in Ts:
    y_s = curve_s(x, T, beta_s)
    y_l = curve_l(x, T, beta_l)

    # First check solid-solid
    line, idmin, idmax = common_tangent(x, y_s, y_s, curve_s, T, beta_s)
    lmin = np.argmin(y_l)
    if line[lmin] <= y_l[lmin]:
        ids = np.absolute([idmin, idmax])
    else:
        line1, idmin1, idmax1 = common_tangent(x, y_s, y_l, curve_s, T, beta_s)
        line2, idmin2, idmax2 = common_tangent(x, y_l, y_s, curve_l, T, beta_l)
        ids = np.absolute([idmin1, idmax1, idmin2, idmax2])
        plot_sl = True
    if (ids < n).all():
        if plot_sl:
            if ids[0] < ids[1]:
                solvus1.append((x[ids[0]], T))
                liquidus.append((x[ids[1]], T))
            if ids[2] < ids[3]:
                liquidus.append((x[ids[2]], T))
                solvus2.append((x[ids[3]], T))
        else:
            if ids[0] < ids[1]:
                solvus1.append((x[ids[0]], T))
                solvus2.append((x[ids[1]], T))

liquidus.sort(key=lambda x: x[0], reverse=False)
solvus1.sort(key=lambda x: x[1], reverse=False)
solvus2.sort(key=lambda x: x[1], reverse=False)

Now that we have the points on the liquidus, solidus, and solvus lines, the following code will plot the phase diagram. You'll see the relevant solid boundaries in blue and the liquidus curve in orange. Dots mark the specific tangent points we calculated.

In [ ]:
plt.rcParams.update({'figure.figsize':(8,6), 'font.size':20, \
                     'lines.linewidth':4, 'lines.markersize':10, 'axes.linewidth':2})
fig, ax = plt.subplots()
ax.plot(*zip(*(max(solvus1), min(solvus2))), 'k', alpha=0.4)
ax.plot(*zip(*solvus1), '-o', c='C0', label='solvus1')
ax.plot(*zip(*liquidus), '-o', c='C1', label='liquidus')
ax.plot(*zip(*solvus2), '-o', c='C0', label='solvus2')
ax.annotate(s=r'$\alpha+\beta$', xy=(0.41, 700))
ax.annotate(s=r'$\alpha$', xy=(0.02, 850))
ax.annotate(s=r'$\beta$', xy=(0.94, 850))
ax.annotate(s=r'$\alpha+L$', xy=(0.12, 920))
ax.annotate(s=r'$L$', xy=(0.43, 1050))
ax.annotate(s=r'$L+\beta$', xy=(0.66, 920))
ax.tick_params(right=True, top=True, direction='in', length=10, width=2)
ax.set_xlim(0, 1)
ax.set_ylim(550, 1250)
ax.set_xlabel(r'$x_{B}$')
ax.set_ylabel(r'$T$ (K)')
plt.show()

## Conclusion
I hope this notebook helped you with visualizing the thermodynamics of regular solutions and eutetic phase diagrams. If you have any remaining questions or ideas for this and other modules, please don't hesitate to reach out.

## Extensions

* If you got this toy version working, can you reproduce a real eutectic system? Data for the Ag-Cu system can be found in [Subramanian, P.R. and Perepezko, J.H. *Journal of Phase Equilibria*, **14**, 1993](https://doi.org/10.1007/BF02652162), but I admittedly couldn't get this to work out right. Pb-Sn is another common eutectic system.

* How might you modify the code to generate the plots for a peritectic system?

## Bonus material

And finally, if you are like undergrad-me and still not convinced that the $\Delta G$ vs. $T$ curves correspond to the eutectic phase diagram, [maybe this animation will change your mind](https://raw.githubusercontent.com/enze-chen/learning_modules/master/fig/eutectic_PD_animation.gif)?